In [1]:
from models.convnext import convnext_base,convnext_large
import torch
model=convnext_large(num_classes=6).cuda()
ckp=torch.load('/home/xyzhou/code/docker_demo/scripts/best.pth')
model.load_state_dict(ckp['model'])

/home/xyzhou/anaconda3/envs/convnext/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

: 

In [ ]:
from models.convnext import convnext_base,convnext_large
import torch
model=convnext_base(num_classes=5).cuda()
ckp=torch.load('/home/xyzhou/code/aisc/traincode/convnext/result/result2/checkpoint-29.pth')
model.load_state_dict(ckp['model'])
model.eval()

In [2]:
modelc=convnext_base(num_classes=4).cuda()
ckp=torch.load('/home/xyzhou/code/aisc/traincode/convnext/result/result_n4/checkpoint-59.pth')
modelc.load_state_dict(ckp['model'])
modelc.eval()

ConvNeXt(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU()
        (pwconv2): Linear(in_features=512, out_features=128, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), g

In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import Compose,ToTensor,Normalize,Resize
transform=Compose([Resize((224,224)),ToTensor(),Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
dataset=ImageFolder('/old_home/xyzhou/aisc/images/train/u2l',transform=transform)
dp=ImageFolder('/old_home/xyzhou/aisc/images/train/u2l')
data_loader = DataLoader(dataset=dataset,batch_size=64,num_workers=4,shuffle=False,drop_last=False)

In [4]:
import numpy as np 
ota,otac=[],[]
with torch.no_grad():
    for i in data_loader:
        outc=modelc.forward_features(i[0].cuda())
        oc=np.array(outc.detach().cpu())
        out=model(i[0].cuda())
        o=np.array(torch.max(out,dim=1).indices.detach().cpu())
        ota.extend(o)
        otac.extend(oc)

In [5]:
from sklearn.cluster import KMeans
X=np.array(otac)
k = KMeans(n_clusters=8, random_state=1).fit(X)
from collections import Counter
a=k.predict(X)
Counter(a)

Counter({4: 750, 2: 397, 1: 349, 6: 144, 5: 418, 0: 133, 7: 403, 3: 105})

In [6]:
from collections import Counter
aota=np.array(ota)
Counter(aota)

Counter({4: 1441, 0: 324, 3: 353, 2: 279, 1: 302})

In [ ]:
np.where(a==2)

In [ ]:
np.where(aota==2)

In [11]:
for i in range(4):
    n=[]
    for j in range(8):
        f2=np.where(aota==i)
        f1=np.where(a==j)
        # if len(set(f1[0])-set(f2[0]))>(len(f1[0])-10) and\
        #     len(set(f1[0])-set(f2[0]))<(len(f1[0])+10):continue
        # else:    
        #     n.append(j)
        # if len(set(f1[0])-set(f2[0]))!=len(f1[0]):n.append(j)
        # n.append(len(set(f1[0])-set(f2[0])))
        n.append((len(set(f1[0])),len(set(f2[0])),len(set(f1[0])-set(f2[0]))))
    print(n)
    # print(f'aota{i} = a{int(np.where(np.array(n)==min(n))[0])} difnum:{min(n)}')

[(133, 324, 133), (349, 324, 349), (397, 324, 73), (105, 324, 105), (750, 324, 750), (418, 324, 418), (144, 324, 144), (403, 324, 403)]
[(133, 302, 132), (349, 302, 349), (397, 302, 397), (105, 302, 105), (750, 302, 750), (418, 302, 117), (144, 302, 144), (403, 302, 403)]
[(133, 279, 133), (349, 279, 334), (397, 279, 397), (105, 279, 105), (750, 279, 486), (418, 279, 418), (144, 279, 144), (403, 279, 403)]
[(133, 353, 133), (349, 353, 349), (397, 353, 397), (105, 353, 103), (750, 353, 749), (418, 353, 418), (144, 353, 144), (403, 353, 53)]


In [ ]:
np.where(aota==3)##对应聚类a3   a:aota:a-aota=415 353 65

In [ ]:
np.where(aota==2)##对应聚类的a2 a:aota:a-aota=728 279 489

In [ ]:
np.where(aota==0)##对应聚类的a4 a:aota:a-aota=394 324 71

In [ ]:
np.where(aota==1)##对应聚类的a5 a:aota:a-aota=430 302 130

In [6]:
from sklearn.cluster import KMeans
X=np.array(ota)
k = KMeans(n_clusters=8, random_state=1).fit(X)

In [17]:
from collections import Counter
a=k.predict(X)
Counter(a)

Counter({2: 728, 4: 394, 1: 190, 5: 430, 0: 226, 7: 184, 3: 415, 6: 132})

In [35]:
np.where(a==4)[0]

array([   1,   13,   38,   50,   65,   67,   71,   72,   82,   84,   86,
         90,   96,   97,   99,  110,  121,  122,  124,  136,  137,  142,
        143,  148,  152,  158,  171,  175,  179,  180,  182,  199,  200,
        201,  202,  203,  206,  207,  208,  209,  210,  212,  224,  226,
        227,  230,  233,  237,  239,  243,  251,  260,  267,  268,  269,
        273,  274,  287,  297,  299,  306,  309,  324,  327,  331,  347,
        352,  360,  361,  362,  364,  370,  384,  391,  392,  406,  408,
        410,  414,  415,  420,  422,  426,  428,  430,  478,  486,  489,
        491,  506,  515,  516,  522,  523,  527,  528,  531,  541,  544,
        554,  573,  629,  632,  634,  638,  641,  645,  653,  675,  676,
        682,  685,  688,  720,  729,  744,  772,  773,  779,  785,  788,
        791,  808,  810,  835,  841,  844,  846,  853,  855,  862,  864,
        868,  870,  873,  894,  896,  898,  902,  905,  910,  915,  919,
        925,  926,  945,  952,  954,  955,  969,  9

In [15]:
np.where(a==2)[0]

NameError: name 'a' is not defined

Unlabeled Classify_n9

In [5]:
kind=set(ota)
for i in kind:
    print(f'kind:{i}  num:{ota.count(i)}')

kind:0  num:392
kind:1  num:622
kind:2  num:1219
kind:3  num:451
kind:5  num:4
kind:6  num:2
kind:7  num:6
kind:8  num:3


In [ ]:
# cluster n9
kind=set(kl)
for i in kind:
    print(f'kind:{i}  num:{kl.count(i)}')

kind:0  num:449
kind:1  num:800
kind:2  num:397
kind:3  num:391
kind:4  num:198
kind:5  num:262
kind:6  num:103
kind:7  num:99


Unlabeled Classify_n4

In [ ]:
kind=set(ota)
for i in kind:
    print(f'kind:{i}  num:{ota.count(i)}')

kind:0  num:396
kind:1  num:674
kind:2  num:1114
kind:3  num:515


In [ ]:
# cluster n5_n4
kind=set(kl)
for i in kind:
    print(f'kind:{i}  num:{kl.count(i)}')

kind:0  num:226
kind:1  num:190
kind:2  num:728
kind:3  num:415
kind:4  num:394
kind:5  num:430
kind:6  num:132
kind:7  num:184


Unlabeled Classify_n5

In [4]:
kind=set(ota)
for i in kind:
    print(f'kind:{i}  num:{ota.count(i)}')

kind:0  num:324
kind:1  num:302
kind:2  num:279
kind:3  num:353
kind:4  num:1441


In [1]:
from torchvision import models
from torchsummary import summary
model=models.resnet50()
summary(model, input_size=[(3,224,224)], batch_size=2, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [2, 64, 112, 112]           9,408
       BatchNorm2d-2          [2, 64, 112, 112]             128
              ReLU-3          [2, 64, 112, 112]               0
         MaxPool2d-4            [2, 64, 56, 56]               0
            Conv2d-5            [2, 64, 56, 56]           4,096
       BatchNorm2d-6            [2, 64, 56, 56]             128
              ReLU-7            [2, 64, 56, 56]               0
            Conv2d-8            [2, 64, 56, 56]          36,864
       BatchNorm2d-9            [2, 64, 56, 56]             128
             ReLU-10            [2, 64, 56, 56]               0
           Conv2d-11           [2, 256, 56, 56]          16,384
      BatchNorm2d-12           [2, 256, 56, 56]             512
           Conv2d-13           [2, 256, 56, 56]          16,384
      BatchNorm2d-14           [2, 256,

In [2]:
from torchsummary import summary
summary(model, input_size=[(3,224,224)], batch_size=2, device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [2, 128, 56, 56]           6,272
         LayerNorm-2           [2, 128, 56, 56]             256
            Conv2d-3           [2, 128, 56, 56]           6,400
         LayerNorm-4           [2, 56, 56, 128]             256
            Linear-5           [2, 56, 56, 512]          66,048
              GELU-6           [2, 56, 56, 512]               0
            Linear-7           [2, 56, 56, 128]          65,664
          Identity-8           [2, 128, 56, 56]               0
             Block-9           [2, 128, 56, 56]               0
           Conv2d-10           [2, 128, 56, 56]           6,400
        LayerNorm-11           [2, 56, 56, 128]             256
           Linear-12           [2, 56, 56, 512]          66,048
             GELU-13           [2, 56, 56, 512]               0
           Linear-14           [2, 56, 